Nombre: Byron Alonso Almeida Quiroz

Materia: Programación Científica y HCP


# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [1]:
import threading
import time
import queue

class Barberia(threading.Thread):
	condicion = threading.Condition() # La condición nos permitirá despiertar/dormir del barbero.
	corteCompleto = threading.Event() # El evento nos ayuda a que todos los elementos la lista CLIENTES hayan sido atendidos.

	def __init__(self, ID):
		threading.Thread.__init__(self)
		self.ID = ID	# Identificación del barbero en el caso de agregar más de 1 barbero. 

	def run(self):
		while True:
			try:
				cliente_actual = salaEspera.get(block=False) # block=false para que el hilo no espere/bloquee para obtener un espacio en la sala de espera.
			except queue.Empty: # Se evalúa si la sala de espera está llena.
				if self.corteCompleto.is_set(): # corteCompleto se actica sólo cuando los elementos de la lista CLIENTES hayan sido atendidos completamente.
					return
				print(f"El barbero {self.ID} está durmiendo...") # Sin elementos en la lista CLIENTES, el barbero va a dormir.
				with self.condicion:
					self.condicion.wait() # El barbero duerme y espera para que el cliente lo despierte.
			else:
				cliente_actual.cortar(self.ID) # Se indica el elemento de la lista de CLIENTES.

class Cliente(threading.Thread):
	
	def __init__(self, ID):
		threading.Thread.__init__(self)
		self.ID = ID # Identificación del elemento en la lista de CLIENTES.

	def cortar(self, id_barbero):  # Se llama a la función cortar desde el hilo Barbería.
		print(f"El barbero {id_barbero} se despierta y empieza a cortar el cabello del cliente {self.ID}")
		time.sleep(2) #	Se define este tiempo el cual demora el barbero en cortar el cabello.
		print(f"El Barbero {id_barbero} terminó de cortar el cabello al cliente {self.ID}")
		print(f"El Barbero {id_barbero} se duerme y aguarda a ser despertado...")
		time.sleep(0.5)  # Se coloca esto para darle tiempo suficiente al barbero para que descanse.
		self.atendido.set() # Se establece el evento como atendido para que el cliente deje la barbería.

	def run(self):
		self.atendido = threading.Event() # Evento mediante el cual se hace posible que el cliente abandone la barbería.

		try:
			salaEspera.put(self, block=False) # Se evalúa si hay espacio en sala de espera.
	 		
		except queue.Full: # Si no hay espacio en la sala de espera se abandona el lugar.
			print(f"La sala de espera está llena, {self.ID} dejó la barbería...")
			return

		print(f"El cliente {self.ID} se sentó en la sala de espera.")
		with Barberia.condicion:
			Barberia.condicion.notify() # Se despierta al barbero para que pueda atender a cliente correspondiente.

		self.atendido.wait() # Se activa la espera para que el cliente reciba su corte.


if __name__ == "__main__":
	CLIENTES = []          # En esta lista se agregan los CLIENTES a ser atendidos.
	salaEspera = queue.Queue(6) # El módulo Queue nos permite manejar una cola en la que se define su tamaño como argumento.

	for i in range(1): # Se habilita el hilo Barberia, en el cual se puede establecer el número de barberos.
		hilo_barbero = Barberia(i)
		hilo_barbero.start()
	
	for i in range(20): # Se Habilita el hilo cliente en el cual se puede establecer el número de clientes deseados.
		time.sleep(1) #Con el time sleep controlamos el tiempo de llegada de los clientes a la barbería
		cliente = Cliente(i) 
		CLIENTES.append(cliente) # Agregamos el número de clientes definido a la lista de CLIENTES.
		cliente.start()

	for cliente in CLIENTES:
		cliente.join()  # Espera a la salida de cada uno de los elementos de la lista CLIENTES que han entrado a la sala.


El barbero 0 está durmiendo...
El cliente 0 se sentó en la sala de espera.
El barbero 0 se despierta y empieza a cortar el cabello del cliente 0
El cliente 1 se sentó en la sala de espera.
El Barbero 0 terminó de cortar el cabello al cliente 0
El Barbero 0 se duerme y aguarda a ser despertado...
El cliente 2 se sentó en la sala de espera.
El barbero 0 se despierta y empieza a cortar el cabello del cliente 1
El cliente 3 se sentó en la sala de espera.
El cliente 4 se sentó en la sala de espera.
El Barbero 0 terminó de cortar el cabello al cliente 1
El Barbero 0 se duerme y aguarda a ser despertado...
El barbero 0 se despierta y empieza a cortar el cabello del cliente 2
El cliente 5 se sentó en la sala de espera.
El cliente 6 se sentó en la sala de espera.
El Barbero 0 terminó de cortar el cabello al cliente 2
El Barbero 0 se duerme y aguarda a ser despertado...
El cliente 7 se sentó en la sala de espera.
El barbero 0 se despierta y empieza a cortar el cabello del cliente 3
El cliente 8 